In [ ]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *

import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections

import hashlib
import os
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5
import base64

In [ ]:
class Client:
    def __init__(self, private_key= None):
        random = Crypto.Random.new().read
        # Generates the private and public keys 1024bit
        if private_key is None:
            self._private_key = RSA.generate(1024, random)
        else:
            self._private_key = RSA.importKey(private_key)
        self._public_key = self._private_key.publickey()
        
    @property
    def identity(self):
        ###
        # The identity is unique to each client and can be made 
        #publicly available. Anybody would be able to send virtual 
        #currency to you using this identity and it will get added 
        #to your wallet.
        ###
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
    
    def to_dict(self):
        d= dict()
        d['private_key']= self._private_key.exportKey('PEM').decode("utf-8")
        return d
    
    def to_string(self, ID= 0):
        s= ['[-----Client %d-----]' % ID,
            '- Private key:' +  self._private_key.exportKey('PEM').decode("utf-8").split('-----')[2],
            '- Public key:' + self._public_key.exportKey('PEM').decode("utf-8").split('-----')[2],
            '- Identity: ' + self.identity
           ]
        return '\n'.join(s)
        

def getClients(fileCSV= 'clients.csv'):
    try:
        df= pd.read_csv(fileCSV)
    except:
        return []
    return [Client(private_key= df['private_key'][i]) for i in range(len(df))]

def createClient(fileCSV= 'clients.csv'):
    try:
        df= pd.read_csv(fileCSV)
    except:
        df= pd.DataFrame(columns= ['private_key'])
    
    client= Client()
    df = df.append(client.to_dict(), ignore_index=True)
    df.to_csv(fileCSV, index= False)

class TabClients(QWidget):  
    
    def __init__(self, parent):
        super(QWidget, self).__init__(parent)
        self.layout = QVBoxLayout(self)
        self.labelList = QLabel("List of Clients")
        self.textEditList = QPlainTextEdit(self)
        self.buttonNewClient = QPushButton("New client")
        self.buttonNewClient.clicked.connect(self.on_clickNewClient)
        
        self.layout.addWidget(self.labelList)
        self.layout.addWidget(self.textEditList)
        self.layout.addWidget(self.buttonNewClient)
        self.setLayout(self.layout)
        
        self.update()
        
    def update(self):
        clients= getClients()
        c= [ clients[i].to_string(i+1) for i in range(len(clients)-1, -1, -1)]
        s= '\n=============================================================\n\n'.join(c)
        self.textEditList.document().setPlainText(s)
        #self.textEditList.setFont(QFont("Arial", 16))
        
    @pyqtSlot()
    def on_clickNewClient(self):
        createClient()
        self.update()
        

In [ ]:
class Transaction:
    #Client can be both a sender or a recipient of the money.
    #Create a transaction and specify your public address in it.
    
    def __init__(self, sender, recipient, value):
        #Initialization of a transaction class
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()
        self.sign = None
        self.confirmed= False
    
    def to_dict(self):
        #Combines all the four above-mentioned instance variables 
        #in a dictionary object.
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender

        return collections.OrderedDict({
            'sender': identity,
            'recipient': self.recipient,
            'value': self.value,
            'time' : self.time,
            'sign' : self.sign,
            'confirmed': self.confirmed
        })
    
    def sign_transaction(self, sender_private_key):
        #The generated signature is decoded to get the ASCII 
        #representation for printing and storing it in our blockchain.
        private_key = sender_private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        self.sign= binascii.hexlify(signer.sign(h)).decode('ascii')
    
    def to_string(self):
        dict = self.to_dict()
        s= []
        for n in dict:
            s.append(n + ': ' + str(dict[n]))
            s.append('-----')
        return '\n'.join(s)

    
def transactionFromDict(dict):
    txn= Transaction(dict['sender'], dict['recipient'], dict['value'])
    txn.time = dict['time']
    txn.sign = dict['sign']
    txn.confirmed= dict['confirmed']
    return txn
    
class SelectionClient(QWidget):
    
    def __init__ (self, parent):
        super(SelectionClient, self).__init__(parent)
        layout = QHBoxLayout()
        self.cb = QComboBox()
        self.clients= getClients()
        i= 0
        for client in self.clients:
            i+= 1
            self.cb.addItem('Clients %d' % i)
        self.clientID= 0
        self.client = self.clients[0]
        self.cb.currentIndexChanged.connect(self.selectionChange)
    
        layout.addWidget(self.cb)
        self.setLayout(layout)
    
    def selectionChange(self, i):
        self.clientID= i
        self.client = self.clients[i]
    
    def update(self):
        self.cb.clear()
        self.clients= getClients()
        i= 0
        for client in self.clients:
            i+= 1
            self.cb.addItem('Clients %d' % i)

class TabCreateTransaction(QWidget):
    
    def __init__ (self, parent):
        super(QWidget, self).__init__(parent)
        self.layout = QVBoxLayout(self)
        
        self.buttonUpdate= QPushButton("Update clients")
        self.buttonUpdate.setFixedWidth(120)
        self.buttonUpdate.clicked.connect(self.on_clickUpdate)
        self.layout.addWidget(self.buttonUpdate)
        
        self.layoutForm= QFormLayout(self)
        self.clientA=  SelectionClient(self)
        self.layoutForm.addRow('Sender: ', self.clientA)
        self.clientB=  SelectionClient(self)
        self.layoutForm.addRow('Recipient: ', self.clientB)
        self.textEditValue = QPlainTextEdit(self)
        self.layoutForm.addRow('Value: ', self.textEditValue)
        self.layout.addLayout(self.layoutForm)
        
        
        self.buttonCreate= QPushButton("Create transaction")
        self.buttonCreate.clicked.connect(self.on_clickCreate)
        self.buttonCreate.setFixedWidth(120)
        self.layout.addWidget(self.buttonCreate)
        
        self.textEditView= QPlainTextEdit(self)
        self.layout.addWidget(self.textEditView)
        
        self.setLayout(self.layout)
    
    def check(self):
        self.sender= self.clientA.client
        self.recipient= self.clientB.client
        if self.sender.identity == self.recipient.identity:
            return '2 client are the same !'
        self.value= self.textEditValue.document().toPlainText()
        self.value= str.encode(self.value)
        try:
            self.value= float(self.value)
        except:
            return 'This value is not a number !'
        
        return None
    
    @pyqtSlot()
    def on_clickCreate(self):
        c= self.check()
        if not c is None:
            self.textEditView.document().setPlainText(c)
            return
        fileCSV= 'transactions.csv'
        tsn= Transaction(self.sender.identity, self.recipient.identity, float(self.value))
        tsn.sign_transaction(self.sender._private_key)
        self.textEditView.document().setPlainText(tsn.to_string())
        try:
            df= pd.read_csv(fileCSV)
        except:
            df= pd.DataFrame(columns= ['sender', 'recipient', 'value', 'time', 'sign', 'confirmed'])
        d= tsn.to_dict()
        df = df.append(d, ignore_index=True)
        df.to_csv(fileCSV, index= False)
    
    @pyqtSlot()
    def on_clickUpdate(self):
        self.clientA.update()
        self.clientB.update()

def getTransactions():
    fileCSV= 'transactions.csv'
    try:
        df= pd.read_csv(fileCSV)
    except:
        return []
    d= df.to_dict(orient= 'records')
    return [transactionFromDict(t) for t in d]

class TabTransactionPool(QWidget):
    
    def __init__ (self, parent):
        super(QWidget, self).__init__(parent)
        self.layout = QVBoxLayout(self)
        self.labelList = QLabel("List of transactions")
        self.textEditList = QPlainTextEdit(self)
        self.buttonUpdate = QPushButton("Update")
        self.buttonUpdate.clicked.connect(self.on_clickUpdate)
        
        self.layout.addWidget(self.labelList)
        self.layout.addWidget(self.textEditList)
        self.layout.addWidget(self.buttonUpdate)
        self.setLayout(self.layout)
        self.update()
    
    def update(self):
        transactions= getTransactions()
        c= [ '[-----Transaction %d-----]\n\n' % i + transactions[i].to_string() for i in range(len(transactions)-1, -1, -1)]
        s= '\n=============================================================\n\n'.join(c)
        self.textEditList.document().setPlainText(s)
        #self.textEditList.setFont(QFont("Arial", 16))
    
    @pyqtSlot()
    def on_clickUpdate(self):
        self.update()

class TabTransactions(QTabWidget):  
    
    def __init__(self, parent):
        super(QWidget, self).__init__(parent)
        self.tabCreate= TabCreateTransaction(self)
        self.addTab(self.tabCreate, 'Creating')
        
        self.tabPool= TabTransactionPool(self)
        self.addTab(self.tabPool, 'Pool')
        
        

In [ ]:
class Block:
    def __init__(self):
        self.verified_transactions = []
        self.previous_block_hash = ""
        self.hash= None #block hash
        self.nonce = ""
        self.difficulty= 10
        self.timestamp = datetime.datetime.now()
        
    def to_dict(self):
        d= dict()
        d['verified_transactions']= self.verified_transactions
        d['previous_block_hash']= self.previous_block_hash
        d['timestamp']= self.timestamp
        d['difficulty']= self.difficulty
        d['nonce']= self.nonce
        d['hash']= self.hash
        return collections.OrderedDict(d)

        
def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()
        
def mine(message, difficulty=1):
    assert difficulty >= 1
    prefix = '1' * difficulty
    i= 0
    while True :
        digest = sha256(str(hash(message)) + str(i))
        if digest.startswith(prefix):
            print ("after " + str(i) + " iterations found nonce: "+ digest)
            return digest
        i+= 1

def getTransactionInPool():
    fileCSV= 'transactions.csv'
    try:
        df= pd.read_csv(fileCSV)
    except:
        return []
    txns= df.to_dict(orient= 'records')
    txns= [transactionFromDict(txn) for txn in txns if txn['confirmed'] == False]
    return txns

def getLastBlockHash():
    fileCSV= 'blocks.csv'
    try:
        df= pd.read_csv(fileCSV)
    except:
        return 'There are no blocks !'
    return df['hash'][-1]

import time        
class TabMine(QWidget):
    
    def __init__ (self, parent):
        super(QWidget, self).__init__(parent)
        self.layout = QVBoxLayout(self)
        self.label = QLabel("Information of new block")
        self.label.setFont(QFont( "lucida", 12, QFont.Bold))
        self.labelList = QLabel("Unconfirmed transactions:")
        self.textEditList = QPlainTextEdit(self)
        self.textEditList.setFixedHeight(220)
        self.buttonNew = QPushButton("Update")
        self.buttonNew.clicked.connect(self.on_clickNew)
        self.labelPreviousBlockHash= QLabel("Previous block hash: ")
        self.textEditPreviousBlockHash = QPlainTextEdit(self)
        self.textEditPreviousBlockHash.setFixedHeight(60)
        
        self.labelMine = QLabel("Mining")
        self.layoutForm= QFormLayout(self)
        self.labelMine.setFont(QFont( "lucida", 12, QFont.Bold))
        self.textEditDifficulty= QPlainTextEdit(self)
        self.textEditDifficulty.document().setPlainText('3')
        self.textEditDifficulty.setFixedHeight(30)
        self.layoutForm.addRow('Difficulty', self.textEditDifficulty)
        self.buttonMine= QPushButton("Mine")
        self.layoutForm.addRow('', self.buttonMine)
        self.buttonMine.clicked.connect(self.on_clickMine)
        self.textEditNonce= QPlainTextEdit(self)
        self.textEditNonce.setFixedHeight(60)
        self.layoutForm.addRow('Nonce', self.textEditNonce)
        
        
        self.layout.addWidget(self.label)
        self.layout.addWidget(self.labelList)
        self.layout.addWidget(self.textEditList)
        self.layout.addWidget(self.labelPreviousBlockHash)
        self.layout.addWidget(self.textEditPreviousBlockHash)
        self.layout.addWidget(self.buttonNew)
        self.layout.addWidget(self.labelMine)
        self.layout.addLayout(self.layoutForm)
        
        
        self.setLayout(self.layout)
        self.new()
    
    def new(self):
        maxTranInBlock= 5
        txns= getTransactionInPool()
        n = min(maxTranInBlock, len(txns))
        self.transactions= txns[:n]
        c= [ '[-----Transaction %d-----]\n\n' % i + self.transactions[i].to_string() for i in range(len(self.transactions)-1, -1, -1)]
        s= '\n=============================================================\n\n'.join(c)
        self.textEditList.document().setPlainText(s)
        self.textEditPreviousBlockHash.document().setPlainText(getLastBlockHash())
        
    @pyqtSlot()
    def on_clickNew(self):
        self.new()
        
    @pyqtSlot()
    def on_clickMine(self):
        self.block= Block()
        self.block.verified_transactions= self.transactions
        for i in range(len(self.block.verified_transactions)):
            self.block.verified_transactions[i].confirmed= True
        difficulty= int(self.textEditDifficulty.document().toPlainText())
        assert difficulty >= 1
        prefix = '1' * difficulty
        i= 0
        while True :
            digest = sha256(str(hash(self.block)) + str(i))
            print(digest)
            if digest.startswith(prefix):
                print ("After " + str(i) + " iterations found nonce: "+ digest)
                self.textEditNonce.document().setPlainText(digest)
                return 
            i+= 1
        self.block.nonce= digest
    
        


In [ ]:
class TabBlock(QTabWidget):  
    
    def __init__(self, parent):
        super(QWidget, self).__init__(parent)
        self.tabMine= TabMine(self)
        self.addTab(self.tabMine, 'Mining')

In [ ]:
class TableWidget(QWidget):
    
    def __init__(self, parent):
        super(QWidget, self).__init__(parent)
        self.layout = QVBoxLayout(self)
        
        # Initialize tab screen
        self.tabs = QTabWidget()
        self.tabClients= TabClients(self)
        self.tabTransactions= TabTransactions(self)
        self.tabBlock= TabBlock(self)
        # Add tabs 
        self.tabs.addTab(self.tabClients,"Clients")
        self.tabs.addTab(self.tabTransactions, "Transantions")
        self.tabs.addTab(self.tabBlock, "Blockchain")
        # Add tabs to widget
        self.layout.addWidget(self.tabs)
        self.setLayout(self.layout)

In [ ]:
class App(QMainWindow):

    def __init__(self):
        super().__init__()
        self.title = 'Blockchain'
        self.left = 0
        self.top = 0
        self.width = 900
        self.height = 1200
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        
        self.table_widget = TableWidget(self)
        self.setCentralWidget(self.table_widget)
        self.show()

In [ ]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    blockchain = App()
    sys.exit(app.exec_())

In [ ]:
getTransactions()

In [1]:
# -*- coding: utf-8 -*-

# Form implementation generated from reading ui file '/tmp/blockchainAH3096.ui'
#
# Created by: PyQt5 UI code generator 5.9.2
#
# WARNING! All changes made in this file will be lost!

from PyQt5 import QtCore, QtGui, QtWidgets

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(800, 924)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.centralwidget)
        self.verticalLayout.setObjectName("verticalLayout")
        self.tabWidget = QtWidgets.QTabWidget(self.centralwidget)
        self.tabWidget.setObjectName("tabWidget")
        self.tabClient = QtWidgets.QWidget()
        self.tabClient.setObjectName("tabClient")
        self.verticalLayout_3 = QtWidgets.QVBoxLayout(self.tabClient)
        self.verticalLayout_3.setObjectName("verticalLayout_3")
        self.tabCreateClient = QtWidgets.QTabWidget(self.tabClient)
        self.tabCreateClient.setObjectName("tabCreateClient")
        self.tabCreate = QtWidgets.QWidget()
        self.tabCreate.setObjectName("tabCreate")
        self.verticalLayout_4 = QtWidgets.QVBoxLayout(self.tabCreate)
        self.verticalLayout_4.setObjectName("verticalLayout_4")
        self.pushButtonGenKey = QtWidgets.QPushButton(self.tabCreate)
        self.pushButtonGenKey.setMaximumSize(QtCore.QSize(145, 16777215))
        self.pushButtonGenKey.setObjectName("pushButtonGenKey")
        self.verticalLayout_4.addWidget(self.pushButtonGenKey)
        self.label = QtWidgets.QLabel(self.tabCreate)
        self.label.setObjectName("label")
        self.verticalLayout_4.addWidget(self.label)
        self.plainTextEditPrivateKey = QtWidgets.QPlainTextEdit(self.tabCreate)
        self.plainTextEditPrivateKey.setObjectName("plainTextEditPrivateKey")
        self.verticalLayout_4.addWidget(self.plainTextEditPrivateKey)
        self.label_2 = QtWidgets.QLabel(self.tabCreate)
        self.label_2.setObjectName("label_2")
        self.verticalLayout_4.addWidget(self.label_2)
        self.plainTextEditPublicKey = QtWidgets.QPlainTextEdit(self.tabCreate)
        self.plainTextEditPublicKey.setObjectName("plainTextEditPublicKey")
        self.verticalLayout_4.addWidget(self.plainTextEditPublicKey)
        self.pushButtonCreateClient = QtWidgets.QPushButton(self.tabCreate)
        self.pushButtonCreateClient.setMaximumSize(QtCore.QSize(145, 16777215))
        self.pushButtonCreateClient.setObjectName("pushButtonCreateClient")
        self.verticalLayout_4.addWidget(self.pushButtonCreateClient)
        self.tabCreateClient.addTab(self.tabCreate, "")
        self.tabList = QtWidgets.QWidget()
        self.tabList.setObjectName("tabList")
        self.verticalLayout_5 = QtWidgets.QVBoxLayout(self.tabList)
        self.verticalLayout_5.setObjectName("verticalLayout_5")
        self.plainTextEditListClient = QtWidgets.QPlainTextEdit(self.tabList)
        self.plainTextEditListClient.setObjectName("plainTextEditListClient")
        self.verticalLayout_5.addWidget(self.plainTextEditListClient)
        self.tabCreateClient.addTab(self.tabList, "")
        self.verticalLayout_3.addWidget(self.tabCreateClient)
        self.tabWidget.addTab(self.tabClient, "")
        self.tabTransaction = QtWidgets.QWidget()
        self.tabTransaction.setObjectName("tabTransaction")
        self.verticalLayout_2 = QtWidgets.QVBoxLayout(self.tabTransaction)
        self.verticalLayout_2.setObjectName("verticalLayout_2")
        self.tabWidget_2 = QtWidgets.QTabWidget(self.tabTransaction)
        self.tabWidget_2.setObjectName("tabWidget_2")
        self.tabCreate1 = QtWidgets.QWidget()
        self.tabCreate1.setObjectName("tabCreate1")
        self.formLayout = QtWidgets.QFormLayout(self.tabCreate1)
        self.formLayout.setObjectName("formLayout")
        self.label_3 = QtWidgets.QLabel(self.tabCreate1)
        self.label_3.setObjectName("label_3")
        self.formLayout.setWidget(0, QtWidgets.QFormLayout.LabelRole, self.label_3)
        self.comboBoxSender = QtWidgets.QComboBox(self.tabCreate1)
        self.comboBoxSender.setMaximumSize(QtCore.QSize(145, 16777215))
        self.comboBoxSender.setObjectName("comboBoxSender")
        self.formLayout.setWidget(0, QtWidgets.QFormLayout.FieldRole, self.comboBoxSender)
        self.label_4 = QtWidgets.QLabel(self.tabCreate1)
        self.label_4.setObjectName("label_4")
        self.formLayout.setWidget(1, QtWidgets.QFormLayout.LabelRole, self.label_4)
        self.plainTextEditSenderID = QtWidgets.QPlainTextEdit(self.tabCreate1)
        self.plainTextEditSenderID.setMaximumSize(QtCore.QSize(16777215, 50))
        self.plainTextEditSenderID.setObjectName("plainTextEditSenderID")
        self.formLayout.setWidget(1, QtWidgets.QFormLayout.FieldRole, self.plainTextEditSenderID)
        self.label_5 = QtWidgets.QLabel(self.tabCreate1)
        self.label_5.setMaximumSize(QtCore.QSize(145, 16777215))
        self.label_5.setObjectName("label_5")
        self.formLayout.setWidget(2, QtWidgets.QFormLayout.LabelRole, self.label_5)
        self.comboBoxRecipent = QtWidgets.QComboBox(self.tabCreate1)
        self.comboBoxRecipent.setMaximumSize(QtCore.QSize(145, 16777215))
        self.comboBoxRecipent.setObjectName("comboBoxRecipent")
        self.formLayout.setWidget(2, QtWidgets.QFormLayout.FieldRole, self.comboBoxRecipent)
        self.label_6 = QtWidgets.QLabel(self.tabCreate1)
        self.label_6.setObjectName("label_6")
        self.formLayout.setWidget(3, QtWidgets.QFormLayout.LabelRole, self.label_6)
        self.plainTextEditRecipientID = QtWidgets.QPlainTextEdit(self.tabCreate1)
        self.plainTextEditRecipientID.setMaximumSize(QtCore.QSize(16777215, 50))
        self.plainTextEditRecipientID.setObjectName("plainTextEditRecipientID")
        self.formLayout.setWidget(3, QtWidgets.QFormLayout.FieldRole, self.plainTextEditRecipientID)
        self.label_7 = QtWidgets.QLabel(self.tabCreate1)
        self.label_7.setObjectName("label_7")
        self.formLayout.setWidget(4, QtWidgets.QFormLayout.LabelRole, self.label_7)
        self.plainTextEditValue = QtWidgets.QPlainTextEdit(self.tabCreate1)
        self.plainTextEditValue.setMaximumSize(QtCore.QSize(200, 40))
        self.plainTextEditValue.setObjectName("plainTextEditValue")
        self.formLayout.setWidget(4, QtWidgets.QFormLayout.FieldRole, self.plainTextEditValue)
        self.pushButtonCreateTran = QtWidgets.QPushButton(self.tabCreate1)
        self.pushButtonCreateTran.setObjectName("pushButtonCreateTran")
        self.formLayout.setWidget(5, QtWidgets.QFormLayout.LabelRole, self.pushButtonCreateTran)
        self.plainTextEditMess = QtWidgets.QPlainTextEdit(self.tabCreate1)
        self.plainTextEditMess.setObjectName("plainTextEditMess")
        self.formLayout.setWidget(6, QtWidgets.QFormLayout.SpanningRole, self.plainTextEditMess)
        self.tabWidget_2.addTab(self.tabCreate1, "")
        self.tabPool = QtWidgets.QWidget()
        self.tabPool.setObjectName("tabPool")
        self.verticalLayout_6 = QtWidgets.QVBoxLayout(self.tabPool)
        self.verticalLayout_6.setObjectName("verticalLayout_6")
        self.plainTextEditPool = QtWidgets.QPlainTextEdit(self.tabPool)
        self.plainTextEditPool.setObjectName("plainTextEditPool")
        self.verticalLayout_6.addWidget(self.plainTextEditPool)
        self.tabWidget_2.addTab(self.tabPool, "")
        self.verticalLayout_2.addWidget(self.tabWidget_2)
        self.tabWidget.addTab(self.tabTransaction, "")
        self.tabBlock = QtWidgets.QWidget()
        self.tabBlock.setObjectName("tabBlock")
        self.verticalLayout_7 = QtWidgets.QVBoxLayout(self.tabBlock)
        self.verticalLayout_7.setObjectName("verticalLayout_7")
        self.tabWidget_3 = QtWidgets.QTabWidget(self.tabBlock)
        self.tabWidget_3.setObjectName("tabWidget_3")
        self.tabCreateBlock = QtWidgets.QWidget()
        self.tabCreateBlock.setObjectName("tabCreateBlock")
        self.formLayout_2 = QtWidgets.QFormLayout(self.tabCreateBlock)
        self.formLayout_2.setObjectName("formLayout_2")
        self.label_8 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_8.setObjectName("label_8")
        self.formLayout_2.setWidget(0, QtWidgets.QFormLayout.LabelRole, self.label_8)
        self.plainTextEditNoTran = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditNoTran.setMaximumSize(QtCore.QSize(150, 30))
        self.plainTextEditNoTran.setObjectName("plainTextEditNoTran")
        self.formLayout_2.setWidget(0, QtWidgets.QFormLayout.FieldRole, self.plainTextEditNoTran)
        self.label_9 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_9.setObjectName("label_9")
        self.formLayout_2.setWidget(1, QtWidgets.QFormLayout.LabelRole, self.label_9)
        self.plainTextEditListUntran = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditListUntran.setObjectName("plainTextEditListUntran")
        self.formLayout_2.setWidget(1, QtWidgets.QFormLayout.FieldRole, self.plainTextEditListUntran)
        self.label_10 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_10.setObjectName("label_10")
        self.formLayout_2.setWidget(4, QtWidgets.QFormLayout.LabelRole, self.label_10)
        self.plainTextEditPreviuosBlock = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditPreviuosBlock.setMaximumSize(QtCore.QSize(16777215, 50))
        self.plainTextEditPreviuosBlock.setObjectName("plainTextEditPreviuosBlock")
        self.formLayout_2.setWidget(4, QtWidgets.QFormLayout.FieldRole, self.plainTextEditPreviuosBlock)
        self.label_11 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_11.setObjectName("label_11")
        self.formLayout_2.setWidget(5, QtWidgets.QFormLayout.LabelRole, self.label_11)
        self.plainTextEditDifficulty = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditDifficulty.setMaximumSize(QtCore.QSize(140, 30))
        self.plainTextEditDifficulty.setObjectName("plainTextEditDifficulty")
        self.formLayout_2.setWidget(5, QtWidgets.QFormLayout.FieldRole, self.plainTextEditDifficulty)
        self.pushButtonUpdateUntran = QtWidgets.QPushButton(self.tabCreateBlock)
        self.pushButtonUpdateUntran.setMaximumSize(QtCore.QSize(200, 16777215))
        self.pushButtonUpdateUntran.setObjectName("pushButtonUpdateUntran")
        self.formLayout_2.setWidget(3, QtWidgets.QFormLayout.FieldRole, self.pushButtonUpdateUntran)
        self.label_12 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_12.setObjectName("label_12")
        self.formLayout_2.setWidget(7, QtWidgets.QFormLayout.LabelRole, self.label_12)
        self.plainTextEditNonce = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditNonce.setMaximumSize(QtCore.QSize(16777215, 30))
        self.plainTextEditNonce.setObjectName("plainTextEditNonce")
        self.formLayout_2.setWidget(7, QtWidgets.QFormLayout.FieldRole, self.plainTextEditNonce)
        self.label_13 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_13.setObjectName("label_13")
        self.formLayout_2.setWidget(2, QtWidgets.QFormLayout.LabelRole, self.label_13)
        self.plainTextEditMerkleRoot = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditMerkleRoot.setMaximumSize(QtCore.QSize(16777215, 60))
        self.plainTextEditMerkleRoot.setObjectName("plainTextEditMerkleRoot")
        self.formLayout_2.setWidget(2, QtWidgets.QFormLayout.FieldRole, self.plainTextEditMerkleRoot)
        self.label_14 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_14.setObjectName("label_14")
        self.formLayout_2.setWidget(8, QtWidgets.QFormLayout.LabelRole, self.label_14)
        self.plainTextEditTime = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditTime.setMaximumSize(QtCore.QSize(16777215, 30))
        self.plainTextEditTime.setObjectName("plainTextEditTime")
        self.formLayout_2.setWidget(8, QtWidgets.QFormLayout.FieldRole, self.plainTextEditTime)
        self.pushButton = QtWidgets.QPushButton(self.tabCreateBlock)
        self.pushButton.setMaximumSize(QtCore.QSize(16777215, 156))
        self.pushButton.setObjectName("pushButton")
        self.formLayout_2.setWidget(6, QtWidgets.QFormLayout.LabelRole, self.pushButton)
        self.label_15 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_15.setObjectName("label_15")
        self.formLayout_2.setWidget(9, QtWidgets.QFormLayout.LabelRole, self.label_15)
        self.plainTextEditBlockHash = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditBlockHash.setMaximumSize(QtCore.QSize(16777215, 90))
        self.plainTextEditBlockHash.setObjectName("plainTextEditBlockHash")
        self.formLayout_2.setWidget(9, QtWidgets.QFormLayout.FieldRole, self.plainTextEditBlockHash)
        self.tabWidget_3.addTab(self.tabCreateBlock, "")
        self.tabBlockchain = QtWidgets.QWidget()
        self.tabBlockchain.setObjectName("tabBlockchain")
        self.formLayout_3 = QtWidgets.QFormLayout(self.tabBlockchain)
        self.formLayout_3.setObjectName("formLayout_3")
        self.label_16 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_16.setObjectName("label_16")
        self.formLayout_3.setWidget(0, QtWidgets.QFormLayout.LabelRole, self.label_16)
        self.plainTextEditIndex = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditIndex.setMaximumSize(QtCore.QSize(145, 30))
        self.plainTextEditIndex.setObjectName("plainTextEditIndex")
        self.formLayout_3.setWidget(0, QtWidgets.QFormLayout.FieldRole, self.plainTextEditIndex)
        self.label_17 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_17.setObjectName("label_17")
        self.formLayout_3.setWidget(1, QtWidgets.QFormLayout.LabelRole, self.label_17)
        self.plainTextEditNoTran_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditNoTran_BC.setMaximumSize(QtCore.QSize(145, 30))
        self.plainTextEditNoTran_BC.setObjectName("plainTextEditNoTran_BC")
        self.formLayout_3.setWidget(1, QtWidgets.QFormLayout.FieldRole, self.plainTextEditNoTran_BC)
        self.label_18 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_18.setObjectName("label_18")
        self.formLayout_3.setWidget(2, QtWidgets.QFormLayout.LabelRole, self.label_18)
        self.label_19 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_19.setObjectName("label_19")
        self.formLayout_3.setWidget(3, QtWidgets.QFormLayout.LabelRole, self.label_19)
        self.plainTextEditListTran = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditListTran.setObjectName("plainTextEditListTran")
        self.formLayout_3.setWidget(2, QtWidgets.QFormLayout.FieldRole, self.plainTextEditListTran)
        self.plainTextEditMerkleRoot_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditMerkleRoot_BC.setMaximumSize(QtCore.QSize(16777215, 60))
        self.plainTextEditMerkleRoot_BC.setObjectName("plainTextEditMerkleRoot_BC")
        self.formLayout_3.setWidget(3, QtWidgets.QFormLayout.FieldRole, self.plainTextEditMerkleRoot_BC)
        self.label_20 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_20.setObjectName("label_20")
        self.formLayout_3.setWidget(4, QtWidgets.QFormLayout.LabelRole, self.label_20)
        self.plainTextEditDifficulty_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditDifficulty_BC.setMaximumSize(QtCore.QSize(145, 30))
        self.plainTextEditDifficulty_BC.setObjectName("plainTextEditDifficulty_BC")
        self.formLayout_3.setWidget(4, QtWidgets.QFormLayout.FieldRole, self.plainTextEditDifficulty_BC)
        self.label_21 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_21.setObjectName("label_21")
        self.formLayout_3.setWidget(5, QtWidgets.QFormLayout.LabelRole, self.label_21)
        self.plainTextEditNonce_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditNonce_BC.setMaximumSize(QtCore.QSize(16777215, 30))
        self.plainTextEditNonce_BC.setObjectName("plainTextEditNonce_BC")
        self.formLayout_3.setWidget(5, QtWidgets.QFormLayout.FieldRole, self.plainTextEditNonce_BC)
        self.label_22 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_22.setObjectName("label_22")
        self.formLayout_3.setWidget(6, QtWidgets.QFormLayout.LabelRole, self.label_22)
        self.plainTextEditTime_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditTime_BC.setMaximumSize(QtCore.QSize(16777215, 30))
        self.plainTextEditTime_BC.setObjectName("plainTextEditTime_BC")
        self.formLayout_3.setWidget(6, QtWidgets.QFormLayout.FieldRole, self.plainTextEditTime_BC)
        self.label_23 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_23.setObjectName("label_23")
        self.formLayout_3.setWidget(7, QtWidgets.QFormLayout.LabelRole, self.label_23)
        self.plainTextEditPreviuosBlock_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditPreviuosBlock_BC.setMaximumSize(QtCore.QSize(16777215, 145))
        self.plainTextEditPreviuosBlock_BC.setObjectName("plainTextEditPreviuosBlock_BC")
        self.formLayout_3.setWidget(7, QtWidgets.QFormLayout.FieldRole, self.plainTextEditPreviuosBlock_BC)
        self.label_24 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_24.setObjectName("label_24")
        self.formLayout_3.setWidget(8, QtWidgets.QFormLayout.LabelRole, self.label_24)
        self.plainTextEditBlockHash_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditBlockHash_BC.setMaximumSize(QtCore.QSize(16777215, 145))
        self.plainTextEditBlockHash_BC.setObjectName("plainTextEditBlockHash_BC")
        self.formLayout_3.setWidget(8, QtWidgets.QFormLayout.FieldRole, self.plainTextEditBlockHash_BC)
        self.pushButtonNextBlock = QtWidgets.QPushButton(self.tabBlockchain)
        self.pushButtonNextBlock.setMaximumSize(QtCore.QSize(130, 16777215))
        self.pushButtonNextBlock.setObjectName("pushButtonNextBlock")
        self.formLayout_3.setWidget(10, QtWidgets.QFormLayout.FieldRole, self.pushButtonNextBlock)
        self.pushButtonPreviousBlock = QtWidgets.QPushButton(self.tabBlockchain)
        self.pushButtonPreviousBlock.setMaximumSize(QtCore.QSize(200, 16777215))
        self.pushButtonPreviousBlock.setObjectName("pushButtonPreviousBlock")
        self.formLayout_3.setWidget(10, QtWidgets.QFormLayout.LabelRole, self.pushButtonPreviousBlock)
        self.tabWidget_3.addTab(self.tabBlockchain, "")
        self.verticalLayout_7.addWidget(self.tabWidget_3)
        self.tabWidget.addTab(self.tabBlock, "")
        self.verticalLayout.addWidget(self.tabWidget)
        MainWindow.setCentralWidget(self.centralwidget)

        self.retranslateUi(MainWindow)
        self.tabWidget.setCurrentIndex(2)
        self.tabCreateClient.setCurrentIndex(0)
        self.tabWidget_2.setCurrentIndex(0)
        self.tabWidget_3.setCurrentIndex(1)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.pushButtonGenKey.setText(_translate("MainWindow", "Generate key"))
        self.label.setText(_translate("MainWindow", "Private key: "))
        self.label_2.setText(_translate("MainWindow", "Public key:"))
        self.pushButtonCreateClient.setText(_translate("MainWindow", "Creating client"))
        self.tabCreateClient.setTabText(self.tabCreateClient.indexOf(self.tabCreate), _translate("MainWindow", "Create client"))
        self.tabCreateClient.setTabText(self.tabCreateClient.indexOf(self.tabList), _translate("MainWindow", "List of clients"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tabClient), _translate("MainWindow", "Client"))
        self.label_3.setText(_translate("MainWindow", "Sender: "))
        self.label_4.setText(_translate("MainWindow", "Identity sender: "))
        self.label_5.setText(_translate("MainWindow", "Recipient: "))
        self.label_6.setText(_translate("MainWindow", "Identity recipient: "))
        self.label_7.setText(_translate("MainWindow", "Value: "))
        self.pushButtonCreateTran.setText(_translate("MainWindow", "Creat transaction"))
        self.tabWidget_2.setTabText(self.tabWidget_2.indexOf(self.tabCreate1), _translate("MainWindow", "Create transaction"))
        self.tabWidget_2.setTabText(self.tabWidget_2.indexOf(self.tabPool), _translate("MainWindow", "Pool"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tabTransaction), _translate("MainWindow", "Transaction"))
        self.label_8.setText(_translate("MainWindow", "No. transaction: "))
        self.label_9.setText(_translate("MainWindow", "List of untransaction: "))
        self.label_10.setText(_translate("MainWindow", "Previous block hash: "))
        self.label_11.setText(_translate("MainWindow", "Difficulty: "))
        self.pushButtonUpdateUntran.setText(_translate("MainWindow", "Update list transaction"))
        self.label_12.setText(_translate("MainWindow", "Nonce: "))
        self.label_13.setText(_translate("MainWindow", "Merkle root: "))
        self.label_14.setText(_translate("MainWindow", "Timestamp: "))
        self.pushButton.setText(_translate("MainWindow", "PoW and Create block"))
        self.label_15.setText(_translate("MainWindow", "Block hash: "))
        self.tabWidget_3.setTabText(self.tabWidget_3.indexOf(self.tabCreateBlock), _translate("MainWindow", "Create block"))
        self.label_16.setText(_translate("MainWindow", "Index: "))
        self.label_17.setText(_translate("MainWindow", "No. transaction: "))
        self.label_18.setText(_translate("MainWindow", "List transaction:"))
        self.label_19.setText(_translate("MainWindow", "Merkle root: "))
        self.label_20.setText(_translate("MainWindow", "Difficulty: "))
        self.label_21.setText(_translate("MainWindow", "Nonce: "))
        self.label_22.setText(_translate("MainWindow", "Timestamp:"))
        self.label_23.setText(_translate("MainWindow", "Previous block: "))
        self.label_24.setText(_translate("MainWindow", "Block hash:"))
        self.pushButtonNextBlock.setText(_translate("MainWindow", "Next block"))
        self.pushButtonPreviousBlock.setText(_translate("MainWindow", "Previous block"))
        self.tabWidget_3.setTabText(self.tabWidget_3.indexOf(self.tabBlockchain), _translate("MainWindow", "Blockchain"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tabBlock), _translate("MainWindow", "Block"))


if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())




SystemExit: 0

/home/ntanh/Anaconda/ENTER/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
